In [ ]:
# Восстановление маркеров на основе данных фотодетектора.

import mne
import numpy as np
import pandas as pd
from google.colab import files

# Имя файла .vhdr
vhdr_path = "Confidence_test000030.vhdr"  # заменить на нужный файл при необходимости

# Загрузка только канала PhotoS
raw = mne.io.read_raw_brainvision(vhdr_path, preload=False)
raw.pick_channels(['PhotoS'])

# Параметры обработки
sfreq = raw.info['sfreq']
duration_sec = raw.n_times / sfreq
chunk_duration = 60  # обработка по 60 секунд
threshold = 2.2      # амплитудный порог
min_interval_sec = 0.5    # мин интервал -> корректировать в зависимости от RT
min_interval_samples = int(min_interval_sec * sfreq)

# Детекция событий по частям
all_markers = []
marker_index = 1

for start_time in range(0, int(duration_sec), chunk_duration):
    end_time = min(start_time + chunk_duration, duration_sec)
    start_sample = int(start_time * sfreq)
    end_sample = int(end_time * sfreq)

    try:
        segment, times = raw[:, start_sample:end_sample]
        signal = segment[0]

        last_index = -min_interval_samples
        for i in range(1, len(signal)):
            if signal[i - 1] < threshold and signal[i] >= threshold:
                if i - last_index >= min_interval_samples:
                    all_markers.append((times[i], marker_index))
                    marker_index += 1
                    last_index = i
    except:
        print(f"Пропущен фрагмент {start_time}–{end_time}")
        continue

# Сохраняем результат
df = pd.DataFrame(all_markers, columns=["onset_sec", "marker_code"])
df.to_csv("photo_markers.csv", index=False)
files.download("photo_markers.csv")

In [ ]:
# Восстановление маркеров уверенности на основании файла с поведенческими данными

import pandas as pd
import glob
import os
import re

def process_type2_markers(vmrk_in: str, base_report_csv: str, vmrk_out: str):
    # 1) Чтение header + Mk строк
    header_lines, raw_marker_lines = [], []
    in_marker_section = False

    with open(vmrk_in, encoding='utf-8') as f:
        for line in f:
            stripped = line.rstrip('\n')
            if stripped.strip() == '[Marker Infos]':
                in_marker_section = True
                header_lines.append(stripped)
                continue

            if not in_marker_section:
                header_lines.append(stripped)
            else:
                if stripped.startswith('Mk'):
                    raw_marker_lines.append(stripped)
                else:
                    header_lines.append(stripped)

    # удаление пропусков
    while header_lines and header_lines[-1].strip() == "":
        header_lines.pop()

    # 2) Парсинг Mk строк
    markers = []
    for mk in raw_marker_lines:
        _, rest = mk.split('=', 1)
        parts = rest.split(',')
        clean = parts[1].replace(' ', '')
        latency = int(parts[2])
        markers.append({'parts': parts, 'latency': latency, 'clean': clean})

    # 3) Загрузка BaseReport CSV (автодетекция , vs ;)
    with open(base_report_csv, 'r', encoding='utf-8', errors='replace') as f:
        first = f.readline()
    sep = ',' if ',' in first else ';'
    df = pd.read_csv(base_report_csv, sep=sep, encoding='utf-8', engine='python')
    df.columns = df.columns.str.strip().str.lstrip('\ufeff')
    conf_col = next(c for c in df.columns if 'Увер' in c)
    conf = df[conf_col].astype(int)

    # 4) Добавление S 6/S 7 посередине S 4/5 и следующего S 1
    injected = []
    s45_idxs = [i for i, m in enumerate(markers) if m['clean'] in ('S4','S5')]
    for trial_idx, idx45 in enumerate(s45_idxs):
        lat45 = markers[idx45]['latency']
        # поиск S 1
        s1s = [m['latency'] for m in markers if m['latency'] > lat45 and m['clean']=='S1']
        if s1s:
            midpoint = lat45 + (min(s1s) - lat45)//2
        else:
            midpoint = lat45 + 500  # 500 ms после последней S4/5

        label = f"S  {6 if conf.iloc[trial_idx] == 1 else 7}"
        injected.append({'parts': ['Stimulus', label, str(midpoint), '1', '0'], 'latency': midpoint})

    # 5) Объединение и сортировка
    all_entries = markers + injected
    all_entries.sort(key=lambda x: x['latency'])

    # 6) Новый .vmrk
    with open(vmrk_out, 'w', encoding='utf-8') as f:
        for line in header_lines:
            f.write(line + "\n")
        for i, ent in enumerate(all_entries, 1):
            f.write(f"Mk{i}=" + ",".join(ent['parts']) + "\n")

    print(f"New Type 2 VMRK: {os.path.basename(vmrk_out)}")

def find_base_report(recording_num: str) -> str:
    # Поиск подходящего BaseReport по названию
    pattern = f"BaseReport_{recording_num}_*CORR*.csv"
    matches = glob.glob(pattern)
    if not matches:
        raise FileNotFoundError(f"No BaseReport CSV found for recording {recording_num}")
    if len(matches) > 1:
        raise FileExistsError(f"Multiple BaseReport CSVs found for recording {recording_num}: {matches}")
    return matches[0]


def main():
    # Обработка всех .vmrk в директории
    for vmrk_path in glob.glob("*.vmrk"):
        # пропуск, если уже есть суффикс
        if vmrk_path.endswith("_ConfMarkers.vmrk"):
            continue

        # Извлечение номера записи, удаляя нули
        match = re.search(r"(\d+)", vmrk_path)
        if not match:
            print(f"Skipping {vmrk_path}: no recording number found.")
            continue
        rec_raw = match.group(1)
        try:
            rec_num = str(int(rec_raw))  # удаление нулей
        except ValueError:
            print(f"Skipping {vmrk_path}: invalid recording number {rec_raw}.")
            continue
        try:
            base_csv = find_base_report(rec_num)
        except (FileNotFoundError, FileExistsError) as e:
            print(f"Skipping {vmrk_path}: {e}")
            continue

        vmrk_out = vmrk_path.replace('.vmrk', '_ConfMarkers.vmrk')
        process_type2_markers(vmrk_path, base_csv, vmrk_out)

if __name__ == '__main__':
    main()

In [ ]:
# Изменение маркера S 1 на основании маркеров правильности и уверенности.

import re
import os
import glob

def process_type1_preserve(vmrk_in, vmrk_out):
    # 1) Чтение header + маркеров
    header_lines = []
    raw_marker_lines = []
    in_marker_section = False

    with open(vmrk_in, encoding='utf-8') as f:
        for line in f:
            if line.strip() == '[Marker Infos]':
                in_marker_section = True
                header_lines.append(line.rstrip('\n'))
                continue

            if not in_marker_section:
                header_lines.append(line.rstrip('\n'))
            else:
                # только строки Mk
                if line.startswith('Mk'):
                    raw_marker_lines.append(line.rstrip('\n'))
                else:
                    header_lines.append(line.rstrip('\n'))

    # 2) Парсинг Mk строк в словарь
    markers = []
    for idx, line in enumerate(raw_marker_lines):
        prefix, rest = line.split('=', 1)
        parts = rest.split(',')
        clean_label = parts[1].replace(' ', '')
        latency = int(parts[2])
        markers.append({
            'idx': idx,
            'prefix': prefix,
            'parts': parts,
            'clean': clean_label,
            'latency': latency
        })

    # 3) Комбинации правильности и уверенности для S1
    combo_map = {
        ('S4','S6'): '11',
        ('S5','S6'): '12',
        ('S4','S7'): '13',
        ('S5','S7'): '14',
    }
    replacements = {}
    for m in markers:
        if m['clean'] == 'S1':
            # следующий маркер точности
            acc = next((x for x in markers if x['latency'] > m['latency'] and x['clean'] in ('S4','S5')), None)
            # следующий маркер уверенности
            conf = next((x for x in markers if x['latency'] > m['latency'] and x['clean'] in ('S6','S7')), None)
            if acc and conf:
                replacements[m['idx']] = combo_map[(acc['clean'], conf['clean'])]

    # 4) Переписывание только S1 маркеров
    out_marker_lines = []
    for idx, m in enumerate(markers):
        parts = m['parts']
        if idx in replacements:
            parts[1] = f"S {replacements[idx]}"
        out_marker_lines.append(f"{m['prefix']}=" + ",".join(parts))

    # 5) Возвращение header и обновленных маркеров
    # удаление лишних пробелов
    while header_lines and header_lines[-1].strip() == "":
        header_lines.pop()

    with open(vmrk_out, 'w', encoding='utf-8') as f:
        for l in header_lines:
            f.write(l + "\n")
        for l in out_marker_lines:
            f.write(l + "\n")

    print(f"Processed: {vmrk_in} -> {vmrk_out}")


if __name__ == '__main__':
    # Обработка всех .vmrk файлов в директории
    pattern = '*.vmrk'
    for vmrk_in in glob.glob(pattern):
        # пропуск файлов уже с суффиксом
        if vmrk_in.endswith('_NewMarkers.vmrk'):
            continue
        base, ext = os.path.splitext(vmrk_in)
        vmrk_out = f"{base}_NewMarkers{ext}"
        process_type1_preserve(vmrk_in, vmrk_out)

In [ ]:
# Преобразование файлов .vmrk в файлы .Markers для импорта в BrainVision Analyzer.

import os
import glob

# настройки
input_dir = ""  # папка с файлами .vmrk
sampling_rate = 1000      # в Hz
sampling_interval = 1     # в ms

# программа
for vmrk_path in glob.glob(os.path.join(input_dir, "*.vmrk")):
    basename = os.path.splitext(os.path.basename(vmrk_path))[0]
    out_fname = f"{basename}_Raw Data.Markers"
    out_path = os.path.join(input_dir, out_fname)

    with open(vmrk_path, "r", encoding="utf-8") as f:
        lines = f.readlines()

    # нахождение начала раздела [Marker Infos]
    try:
        mi_idx = lines.index("[Marker Infos]\n")
    except ValueError:
        print(f"Skipping {vmrk_path}: no [Marker Infos] section found.")
        continue

    # всё после [Marker Infos] - это маркеры
    markers = []
    for line in lines[mi_idx+1:]:
        line = line.strip()
        if not line or line.startswith(";"):
            continue
        if not line.startswith("Mk"):
            continue

        # разделение данных
        _, data = line.split("=", 1)
        parts = data.split(",", 5)
        # части: [Type, Description, Position, Length, Channel, (extra if present)]
        if parts[0].lower() == "new segment":
            # пропуск системной строки "Mk1=New Segment"
            continue

        typ         = parts[0]
        desc        = parts[1].replace("\\1", ",")
        position    = parts[2]
        length      = parts[3] if len(parts) > 3 else ""
        channel_num = parts[4] if len(parts) > 4 else "0"

        channel = "All" if channel_num == "0" else channel_num
        markers.append(f"{typ},{desc},{position},{length},{channel}")

    # Создание файла .Markers
    with open(out_path, "w", encoding="utf-8") as out:
        out.write(f"Sampling rate: {sampling_rate}Hz, SamplingInterval: {sampling_interval}ms\n")
        out.write("Type, Description, Position, Length, Channel\n")
        for m in markers:
            out.write(m + "\n")

    print(f"Written: {out_path}")

In [ ]:
# Изменение маркеров S 2 и S 3 в соответствии с маркером S 1 для файлов .Markers.

import os
import glob
import re

def process_three_stages(markers_in, markers_out):
    # Чтение строк
    with open(markers_in, 'r', encoding='utf-8') as f:
        lines = [line.rstrip('\n') for line in f]

    # Разделение header и маркеров
    # Первые 2 ряда системные
    header_lines = lines[:2]
    marker_lines = lines[2:]

    # Парсинг маркеров в словари
    records = []
    for line in marker_lines:
        if not line.strip():
            continue
        parts = line.split(',')  # ['Stimulus', 'S  2', '38131', '1', 'All']
        clean = parts[1].replace(' ', '')  # 'S2'
        records.append({'parts': parts, 'clean': clean})

    # Поиск индексов S11-S14
    group_idxs = [i for i, r in enumerate(records) if r['clean'] in ('S11','S12','S13','S14')]

    # Обработка по сегментам
    for idx_num, start in enumerate(group_idxs):
        end = group_idxs[idx_num+1] if idx_num+1 < len(group_idxs) else len(records)
        group_label = records[start]['clean']  # e.g. 'S11'
        suffix = group_label[-1]  # '1', '2', '3', or '4'
        for r in records[start+1:end]:
            if r['clean'] == 'S2':
                # Меняем на 2X
                r['parts'][1] = f"S {2}{suffix}"
            elif r['clean'] == 'S3':
                # Меняем на S3X
                r['parts'][1] = f"S {3}{suffix}"

    # Преобразование строк
    out_marker_lines = [','.join(r['parts']) for r in records]

    # Вывод
    with open(markers_out, 'w', encoding='utf-8') as f:
        for line in header_lines:
            f.write(line + '\n')
        for line in out_marker_lines:
            f.write(line + '\n')

    print(f"Processed: {markers_in} -> {markers_out}")

if __name__ == '__main__':
    # Директория для вывода
    out_dir = 'three_stages'
    os.makedirs(out_dir, exist_ok=True)

    # Обработка всех файлов .Markers в директории
    for infile in glob.glob('*.Markers'):
        if infile.endswith('_three_stages.Markers'):
            continue
        base, ext = os.path.splitext(infile)
        outfile = os.path.join(out_dir, f"{base}_three_stages{ext}")
        process_three_stages(infile, outfile)